In [455]:
import json
import matplotlib.pyplot as plt
import numpy as np

DATA_PATH = 'workdir/data/contest_input_filtered.json'

json_file = json.load(open(DATA_PATH, 'r'))

In [539]:
orders

{'10001': {'order_id': 10001,
  'pickup_point_id': 40001,
  'pickup_from': 480,
  'pickup_to': 570,
  'pickup_location_x': 284,
  'pickup_location_y': 235,
  'dropoff_point_id': 60001,
  'dropoff_from': 600,
  'dropoff_to': 960,
  'dropoff_location_x': 252,
  'dropoff_location_y': 197,
  'payment': 313},
 '10003': {'order_id': 10003,
  'pickup_point_id': 40003,
  'pickup_from': 390,
  'pickup_to': 1020,
  'pickup_location_x': 225,
  'pickup_location_y': 141,
  'dropoff_point_id': 60003,
  'dropoff_from': 1050,
  'dropoff_to': 1080,
  'dropoff_location_x': 115,
  'dropoff_location_y': 38,
  'payment': 380},
 '10005': {'order_id': 10005,
  'pickup_point_id': 40005,
  'pickup_from': 300,
  'pickup_to': 780,
  'pickup_location_x': 233,
  'pickup_location_y': 108,
  'dropoff_point_id': 60005,
  'dropoff_from': 840,
  'dropoff_to': 870,
  'dropoff_location_x': 12,
  'dropoff_location_y': 101,
  'payment': 482},
 '10008': {'order_id': 10008,
  'pickup_point_id': 40008,
  'pickup_from': 360,
 

In [456]:
couriers = json_file['couriers']

In [1044]:
orders = json_file['orders']
orders_pickup_times_0 = np.array([(order['pickup_from'], order['pickup_to']) for order in orders.values()])
orders_pickup_times = np.array([(((order['pickup_from'])), ((order['pickup_to']))) for order in orders.values()])
orders_drop_times = np.array([(((order['dropoff_from'])), ((order['dropoff_to']))) for order in orders.values()])
orders_from = np.array([(order['pickup_location_x'], order['pickup_location_y']) for order in orders.values()])
orders_to = np.array([(order['dropoff_location_x'], order['dropoff_location_y']) for order in orders.values()])
payments = np.array([(order['payment']) for order in orders.values()])

delivery_dists = np.abs(orders_to - orders_from).sum(axis=1)

In [1045]:
ord_from = [(a, b) for a, b in orders_from]
ord_to = [(a, b) for a, b in orders_to]

In [1046]:
def plot_moves(x, y, cur_ids):
    from_points = orders_from[cur_ids]
    to_points = orders_from[cur_ids]

    plt.plot([x], [y], '*')
    plt.plot(from_points[:, 0], from_points[:, 1], '.', color='red')
#    plt.plot(to_points[:, 0], to_points[:, 1], '.', color='blue')  

In [1047]:
def from_point_xy(i):
    return orders_from[i]

def to_point_xy(i):
    return orders_to[i]

In [1048]:
def single_step(x, y, used, cur_time=360, R=120, n_orders=8):
    dists = manhattan(orders_from, (x, y))
    dists_to = delivery_dists
    
    time_to_arrive = cur_time + dists + 10
    
    can_arrive = time_to_arrive < orders_pickup_times[:, 1]
    can_arrive_before = time_to_arrive + 10 > orders_pickup_times[:, 0]
    
    can_deliever = time_to_arrive + delivery_dists < orders_drop_times[:, 1]
    
    can_arrive_2 = can_arrive * can_arrive_before * can_deliever
    
    if np.all(can_arrive == False):
        return []
    
    near_dists = can_arrive_2 * dists + 1000000 * (1 - can_arrive)
    
    if np.all(near_dists == 1000000):
        return []
    
    pts = np.argsort(near_dists)[:n_orders]

    return pts

In [1061]:
def manhattan(vector, point):
    return np.abs((vector - point)).sum(axis=1)

def courier_moves(x, y, cur_time, cur_step=0, used=None, max_step=15, n_vals=10):
    if cur_step > max_step:
        return np.zeros(0)

    current_to_use = single_step(x, y, used, cur_time, n_orders=n_vals)
    resulted_ids = list(current_to_use.copy())
    for cur_order in current_to_use:
        if used[cur_order] == 1:
            continue
        else:
            used[cur_order] = 1
            cur_x_f = orders_from[cur_order, 0]
            cur_y_f = orders_from[cur_order, 1]
            
            cur_x = orders_to[cur_order, 0]
            cur_y = orders_to[cur_order, 1]
            dist_manh = abs(cur_x - x) + abs(cur_y - y) #+ abs(cur_x_f - x) + abs(cur_y_f - y)
            
            resulted_ids = list(resulted_ids) + list(courier_moves(cur_x, cur_y, cur_time + dist_manh, cur_step + 1, used, max_step, max(1, n_vals // 2)))

    return resulted_ids

In [1062]:
results = []

for cour in couriers:
    cou_x = cour['location_x']
    cou_y = cour['location_y']
    
    is_used = np.zeros(len(orders))
    cour = courier_moves(cou_x, cou_y, 360, used=is_used)
    results.append((cou_x, cou_y, cour))
    
    break

In [1063]:
len(set(cour))

190

In [1064]:
inters = []

for _, _, i in results:
    for _, _, j in results:
        inters.append(len(set(i).intersection(j)))

In [1083]:
def use_extra_orders(x, y, order_id):
    from_x, from_y = orders_from[order_id]
    to_x, to_y = orders_to[order_id]
    
    from_dist = abs(from_x - x) + abs(from_y - y)
    to_dist = abs(from_x - to_x) + abs(from_y - to_y)
    can_pick = (360 + from_dist + 10) < orders_pickup_times[order_id, 1]
        
    drop_time = 360 + max(from_dist, orders_pickup_times[order_id, 0]) + to_dist + 10
    can_drop = drop_time + 10 < orders_drop_times[order_id, 1]
    
    return can_drop and can_pick

In [1084]:
for i, aaa in enumerate(results):
    _, _, cour = aaa
    courier = couriers[i]
    
    cour_x = courier['location_x']
    cour_y = courier['location_y']
    cr = list(set(cour))
    new_cr = [cr_val for cr_val in cr if use_extra_orders(cour_x, cour_y, cr_val)]
    
    print(len(cr), len(new_cr))
    
    
    ords = np.array(list(orders.values()))[new_cr]

    o = open(f'outs/{i}.json', 'w')
    
    result_json = {'orders': ords.tolist(), 'couriers': [courier]}
    
    json.dump(result_json, o)
    
    o.close()
    
    break

190 105


In [1086]:
ords

array([{'order_id': 16035, 'pickup_point_id': 46035, 'pickup_from': 390, 'pickup_to': 1110, 'pickup_location_x': 74, 'pickup_location_y': 126, 'dropoff_point_id': 66035, 'dropoff_from': 1140, 'dropoff_to': 1200, 'dropoff_location_x': 289, 'dropoff_location_y': 63, 'payment': 439},
       {'order_id': 16551, 'pickup_point_id': 46551, 'pickup_from': 300, 'pickup_to': 900, 'pickup_location_x': 175, 'pickup_location_y': 199, 'dropoff_point_id': 66551, 'dropoff_from': 819, 'dropoff_to': 960, 'dropoff_location_x': 229, 'dropoff_location_y': 77, 'payment': 398},
       {'order_id': 16042, 'pickup_point_id': 46042, 'pickup_from': 300, 'pickup_to': 900, 'pickup_location_x': 148, 'pickup_location_y': 95, 'dropoff_point_id': 66042, 'dropoff_from': 763, 'dropoff_to': 930, 'dropoff_location_x': 212, 'dropoff_location_y': 255, 'payment': 314},
       {'order_id': 16557, 'pickup_point_id': 46557, 'pickup_from': 360, 'pickup_to': 870, 'pickup_location_x': 232, 'pickup_location_y': 115, 'dropoff_point_

In [1090]:
for i in ords:
    cour_x = couriers[0]['location_x']
    cour_y = couriers[0]['location_y']
    
    dist_to_order = abs(cour_x - i['pickup_location_x']) + abs(cour_y - i['pickup_location_y'])
    dist_to_deliver = abs(i['dropoff_location_x'] - i['pickup_location_x']) + abs(i['dropoff_location_y'] - i['pickup_location_y'])
    
    print(f'start time: {360} need pick: [{i["pickup_from"]}, {i["pickup_to"]}] need deliver: [{i["dropoff_from"]}, {i["dropoff_to"]}]')
    print(f'start time: {360} can pick: {360 + dist_to_order} can deliver: {360 + dist_to_order + 10 + dist_to_deliver}')

start time: 360 need pick: [390, 1110] need deliver: [1140, 1200]
start time: 360 can pick: 518 can deliver: 806
start time: 360 need pick: [300, 900] need deliver: [819, 960]
start time: 360 can pick: 618 can deliver: 804
start time: 360 need pick: [300, 900] need deliver: [763, 930]
start time: 360 can pick: 487 can deliver: 721
start time: 360 need pick: [360, 870] need deliver: [820, 900]
start time: 360 can pick: 591 can deliver: 715
start time: 360 need pick: [300, 840] need deliver: [764, 900]
start time: 360 can pick: 556 can deliver: 719
start time: 360 need pick: [360, 900] need deliver: [930, 960]
start time: 360 can pick: 679 can deliver: 794
start time: 360 need pick: [390, 810] need deliver: [765, 810]
start time: 360 can pick: 608 can deliver: 627
start time: 360 need pick: [330, 870] need deliver: [900, 960]
start time: 360 can pick: 628 can deliver: 843
start time: 360 need pick: [330, 1110] need deliver: [1110, 1140]
start time: 360 can pick: 600 can deliver: 775
star

In [1069]:
ff = list(set(results[0][2]))

In [1070]:
np.array(list(orders.values()))[ff]

array([{'order_id': 16543, 'pickup_point_id': 46543, 'pickup_from': 330, 'pickup_to': 870, 'pickup_location_x': 263, 'pickup_location_y': 146, 'dropoff_point_id': 66543, 'dropoff_from': 818, 'dropoff_to': 930, 'dropoff_location_x': 115, 'dropoff_location_y': 226, 'payment': 511},
       {'order_id': 16035, 'pickup_point_id': 46035, 'pickup_from': 390, 'pickup_to': 1110, 'pickup_location_x': 74, 'pickup_location_y': 126, 'dropoff_point_id': 66035, 'dropoff_from': 1140, 'dropoff_to': 1200, 'dropoff_location_x': 289, 'dropoff_location_y': 63, 'payment': 439},
       {'order_id': 15525, 'pickup_point_id': 45525, 'pickup_from': 360, 'pickup_to': 750, 'pickup_location_x': 288, 'pickup_location_y': 155, 'dropoff_point_id': 65525, 'dropoff_from': 715, 'dropoff_to': 840, 'dropoff_location_x': 11, 'dropoff_location_y': 105, 'payment': 490},
       {'order_id': 16037, 'pickup_point_id': 46037, 'pickup_from': 390, 'pickup_to': 810, 'pickup_location_x': 272, 'pickup_location_y': 104, 'dropoff_point

In [1071]:
orders_drop_times[2992]

array([669, 780])

In [1072]:
is_used = np.zeros(len(orders))
single_step(111, 5, is_used)

array([3817, 3500, 3502, 3503, 1885, 3506, 3510, 1881])